In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fragrantica/Fragrantica Dataset.xlsx
/kaggle/input/dataset-temp/first_100_rows.csv
/kaggle/input/dataset-temp/Normalized_Data (1).csv
/kaggle/input/dataset-temp/GeneratedData (2).xlsx


In [3]:
import pandas as pd
import re

In [5]:
dataset = pd.read_excel('/kaggle/input/fragrantica/Fragrantica Dataset.xlsx')

In [6]:
print(dataset['accords'].dtype)

object


In [7]:
df = pd.DataFrame()

# Extract the relevant columns
df['id'] = dataset['perfume_id']
df['title'] = dataset['title']
df['accords'] = dataset['accords']
df['average_rating'] = dataset['average_rating_']
df['release_date'] = dataset['date']


In [8]:
def list_to_comma_separated_string(input_list):
    return ','.join(map(str.lower, input_list))

In [9]:
# Initialize lists to store scent notes for each perfume
top_notes_list = []
middle_notes_list = []
base_notes_list = []

# Iterate over each row in the DataFrame
for index, row in dataset.iterrows():
    top_notes = []
    middle_notes = []
    base_notes = []
    
    # Iterate over each column from 0 to 65
    for col in range(66):
        note = row[str(col)]  # Get the value in the current column
        
        # Check if the note is not empty or null
        if pd.notnull(note) and note != 'nan':
            category, _, scent = re.split(r'(\d+)', note)  # Split the note into category and scent
            
            if scent == "nan" or scent=='nan' or scent=="":
                continue
            # Append the scent to the respective list based on the category
            if category == 'Top':
                top_notes.append(scent)
            elif category == 'Middle':
                middle_notes.append(scent)
            elif category == 'Base':
                base_notes.append(scent)
    
    # Append the lists of scent notes for the current perfume to the respective master lists
    top_notes_list.append(list_to_comma_separated_string(top_notes))
    middle_notes_list.append(list_to_comma_separated_string(middle_notes))
    base_notes_list.append(list_to_comma_separated_string(base_notes))
    

# Add lists of scent notes to the new DataFrame
df['top_notes'] = top_notes_list
df['middle_notes'] = middle_notes_list
df['base_notes'] = base_notes_list


KeyError: '0'

In [16]:
# Calculate max count for each preference type
total_love_count = dataset['clslove'].max()
total_like_count = dataset['clslike'].max()
total_dislike_count = dataset['clsdislike'].max()

# Normalize user preferences
df['love_normalized'] = dataset['clslove'] / total_love_count
df['like_normalized'] = dataset['clslike'] / total_like_count
df['dislike_normalized'] = dataset['clsdislike'] / total_dislike_count

In [17]:
# Calculate total counts for each season
total_winter_count = dataset['clswinter'].max()
total_spring_count = dataset['clsspring'].max()
total_summer_count = dataset['clssummer'].max()
total_autumn_count = dataset['clsautumn'].max()

# Normalize seasonal preferences
df['normalized_winter'] = dataset['clswinter'] / total_winter_count
df['normalized_spring'] = dataset['clsspring'] / total_spring_count
df['normalized_summer'] = dataset['clssummer'] / total_summer_count
df['normalized_autumn'] = dataset['clsautumn'] / total_autumn_count


In [18]:
# Calculate total counts for daytime and nighttime preferences
total_day_count = dataset['clsday'].max()
total_night_count = dataset['clsnight'].max()

# Normalize DayOrNightPreferences
df['normalized_day'] = dataset['clsday'] / total_day_count
df['normalized_night'] = dataset['clsnight'] / total_night_count


In [19]:
for col in dataset.columns:
    if col.startswith('cls') and '_' in col:
    # Split column name to extract gender and age information
        parts = col.split('_')
        preference_type = parts[0][3:]
        match = re.match(r'([a-zA-Z]+)(\d+(older|under))', parts[1])
        gender = match.group(1)
        age = match.group(2)
        # Aggregate preferences based on gender, age, and preference type
        if preference_type in ['love', 'like', 'dislike']:
            total_count = dataset[col].max()
            normalized_count = dataset[col] / total_count
            new_col_name = f'{preference_type}_{gender}_{age}_normalized'
            df[new_col_name] = normalized_count

        # Aggregate seasonal preferences
        elif preference_type in ['winter', 'spring', 'summer', 'autumn']:
            total_count = dataset[col].max()
            normalized_count = dataset[col] / total_count
            new_col_name = f'{gender}_{age}_{preference_type}_normalized'
            df[new_col_name] = normalized_count

        # Aggregate day/night preferences
        elif preference_type in ['day', 'night']:
            total_count = dataset[col].max()
            normalized_count = dataset[col] / total_count
            new_col_name = f'{gender}_{age}_{preference_type}_normalized'
            df[new_col_name] = normalized_count


In [20]:
df.head

<bound method NDFrame.head of        id                                              title  \
0   44152        Aamal The Spirit of Dubai for women and men   
1   19425                     Aatifa Ajmal for women and men   
2   23831           AA Al-Jazeera Perfumes for women and men   
3   49977  aarewasser Art of Scent - Swiss Perfumes for w...   
4   27823    Aaliyah Hamidi Oud & Perfumes for women and men   
..    ...                                                ...   
95  15056                 C-Thru Blooming Sarantis for women   
96  23471  Charlotte Aux Fraises Adopt' by Reserve Nature...   
97  16635                Strawberry Creme Candie's for women   
98  38744  Voodoo Love Bourbon French Parfums for women a...   
99  14350            Strawberry Starfruit Fruttini for women   

                                              accords  average_rating  \
0          woody,earthy,animalic,amber,musky,balsamic            5.00   
1                  fresh spicy,woody,musky,rose,amber  

In [21]:
import pandas as pd

# Assuming you have a DataFrame named df
column_names = df.columns
column_names_df = pd.DataFrame(column_names, columns=['Column Names'])

# Specify the file path where you want to save the CSV file
csv_file_path = 'column_names.csv'

# Save the column names DataFrame to a CSV file
column_names_df.to_csv(csv_file_path, index=False)

print(f"Column names have been saved to '{csv_file_path}'.")


Column names have been saved to 'column_names.csv'.


In [22]:
import pandas as pd

# Assuming df is your original dataframe
fixed_categories = ['id', 'title', 'release_date', 'accords', 'average_rating', 'top_notes', 'middle_notes', 'base_notes']
# Define categories
categories = ['love', 'like', 'dislike', 'winter', 'summer', 'spring', 'autumn', '25under', '25older', 'day', 'night', 'male', 'female']

# Create new columns for each category
for category in categories:
    # Filter columns containing the category in their name
    if category in ['like']:
        category_columns = [col for col in df.columns if category in col and 'dislike' not in col]
    elif category in ['male']:
        category_columns = [col for col in df.columns if category in col and 'female' not in col]
    else:
        category_columns = [col for col in df.columns if category in col]
    print(category_columns)
    
    # Sum values across these columns along the rows
    df[category] = df[category_columns].sum(axis=1)
# Create a new dataframe with only 'id' and the newly created columns
new_df = df[fixed_categories + categories]

# Display the new dataframe
print(new_df)


['love_normalized', 'love_female_25under_normalized', 'love_male_25under_normalized', 'love_female_25older_normalized', 'love_male_25older_normalized']
['like_normalized', 'like_female_25under_normalized', 'like_male_25under_normalized', 'like_female_25older_normalized', 'like_male_25older_normalized']
['dislike_normalized', 'dislike_female_25under_normalized', 'dislike_male_25under_normalized', 'dislike_female_25older_normalized', 'dislike_male_25older_normalized']
['normalized_winter', 'female_25under_winter_normalized', 'male_25under_winter_normalized', 'female_25older_winter_normalized', 'male_25older_winter_normalized']
['normalized_summer', 'female_25under_summer_normalized', 'male_25under_summer_normalized', 'female_25older_summer_normalized', 'male_25older_summer_normalized']
['normalized_spring', 'female_25under_spring_normalized', 'male_25under_spring_normalized', 'female_25older_spring_normalized', 'male_25older_spring_normalized']
['normalized_autumn', 'female_25under_autum

In [23]:

final_df = new_df.copy()

# Renaming columns in the copied DataFrame
final_df.rename(columns={'release_date': 'date',
                       'average_rating': 'rating',
                       '25under': 'youth',
                       '25older': 'adult'}, inplace=True)

# Display the modified DataFrame
print(final_df)


       id                                              title  date  \
0   44152        Aamal The Spirit of Dubai for women and men  2017   
1   19425                     Aatifa Ajmal for women and men  2014   
2   23831           AA Al-Jazeera Perfumes for women and men     0   
3   49977  aarewasser Art of Scent - Swiss Perfumes for w...  2010   
4   27823    Aaliyah Hamidi Oud & Perfumes for women and men     0   
..    ...                                                ...   ...   
95  15056                 C-Thru Blooming Sarantis for women  2012   
96  23471  Charlotte Aux Fraises Adopt' by Reserve Nature...     0   
97  16635                Strawberry Creme Candie's for women  2012   
98  38744  Voodoo Love Bourbon French Parfums for women a...     0   
99  14350            Strawberry Starfruit Fruttini for women  2009   

                                              accords  rating  \
0          woody,earthy,animalic,amber,musky,balsamic    5.00   
1                  fresh spic

In [24]:
# Assuming df is your DataFrame
final_df.to_excel('finalDataset.xlsx', index=False)  # Set index=False if you don't want to include row numbers


In [25]:
normalize_columns = ['love', 'like', 'dislike', 'summer', 'winter', 'autumn', 'spring', 'day', 'night', 'youth', 'adult', 'male', 'female']
normalize_df = final_df.copy()
for column_name in normalize_columns : 
    maxi =final_df[column_name].max()
    normalize_df[column_name] = final_df[column_name]/maxi
    print(normalize_df[column_name])

0     0.736851
1     0.733263
2     0.026501
3     0.026501
4     0.026501
        ...   
95    0.026501
96    0.119779
97    0.330502
98    0.729676
99    0.533981
Name: love, Length: 100, dtype: float64
0     0.024459
1     0.572043
2     0.024459
3     0.024459
4     0.024459
        ...   
95    0.846791
96    0.771501
97    0.844282
98    0.109788
99    0.830180
Name: like, Length: 100, dtype: float64
0     0.028283
1     0.173352
2     0.028283
3     0.028283
4     0.028283
        ...   
95    0.272610
96    0.280245
97    0.146629
98    0.074255
99    0.200075
Name: dislike, Length: 100, dtype: float64
0     0.614061
1     0.494061
2     0.022061
3     0.814061
4     0.814061
        ...   
95    0.519394
96    0.372848
97    0.246424
98    0.422061
99    0.639030
Name: summer, Length: 100, dtype: float64
0     0.469185
1     0.694781
2     0.024502
3     0.024502
4     0.619416
        ...   
95    0.051543
96    0.203062
97    0.065064
98    0.426119
99    0.024502
Name: wint

In [26]:
final_normalized_df = normalize_df.copy()

#normalize seasons
seasons = ['winter', 'summer', 'autumn', 'spring']
sumSeason = normalize_df[seasons].max(axis=1)
for season in seasons :
    print(sumSeason)
    final_normalized_df[season] = normalize_df[season]/sumSeason
    print(final_normalized_df[season])

#normalize day & night
dayNight = ['day', 'night']
sumDay = normalize_df[dayNight].max(axis=1)
for typee in dayNight: 
    print(sumDay)
    final_normalized_df[typee] = normalize_df[typee]/sumDay
    print(final_normalized_df[typee])

    
#normalize age Type
ageType = ['adult', 'youth']
sumAge = normalize_df[ageType].max(axis=1)
for ageTy in ageType:
    print(ageType)
    final_normalized_df[ageTy] = normalize_df[ageTy]/sumAge
    print(final_normalized_df[ageTy])

    
#normalize gender Type
genderType = ['male', 'female']
sumGender = normalize_df[genderType].max(axis=1)
for genderTy in genderType:
    print(sumGender)
    final_normalized_df[genderTy] = normalize_df[genderTy]/sumGender
    print(final_normalized_df[genderTy])
    
    
print(final_normalized_df.head())

0     0.614061
1     0.694781
2     0.026331
3     0.814061
4     0.814061
        ...   
95    0.571899
96    0.372848
97    0.537964
98    0.426119
99    0.692884
Length: 100, dtype: float64
0     0.764069
1     1.000000
2     0.930545
3     0.030098
4     0.760896
        ...   
95    0.090127
96    0.544623
97    0.120945
98    1.000000
99    0.035362
Name: winter, Length: 100, dtype: float64
0     0.614061
1     0.694781
2     0.026331
3     0.814061
4     0.814061
        ...   
95    0.571899
96    0.372848
97    0.537964
98    0.426119
99    0.692884
Length: 100, dtype: float64
0     1.000000
1     0.711102
2     0.837828
3     1.000000
4     1.000000
        ...   
95    0.908192
96    1.000000
97    0.458068
98    0.990477
99    0.922276
Name: summer, Length: 100, dtype: float64
0     0.614061
1     0.694781
2     0.026331
3     0.814061
4     0.814061
        ...   
95    0.571899
96    0.372848
97    0.537964
98    0.426119
99    0.692884
Length: 100, dtype: float64
0     0

In [27]:
print(final_normalized_df)

       id                                              title  date  \
0   44152        Aamal The Spirit of Dubai for women and men  2017   
1   19425                     Aatifa Ajmal for women and men  2014   
2   23831           AA Al-Jazeera Perfumes for women and men     0   
3   49977  aarewasser Art of Scent - Swiss Perfumes for w...  2010   
4   27823    Aaliyah Hamidi Oud & Perfumes for women and men     0   
..    ...                                                ...   ...   
95  15056                 C-Thru Blooming Sarantis for women  2012   
96  23471  Charlotte Aux Fraises Adopt' by Reserve Nature...     0   
97  16635                Strawberry Creme Candie's for women  2012   
98  38744  Voodoo Love Bourbon French Parfums for women a...     0   
99  14350            Strawberry Starfruit Fruttini for women  2009   

                                              accords  rating  \
0          woody,earthy,animalic,amber,musky,balsamic    5.00   
1                  fresh spic

In [28]:

# # Assuming final_normalized_df is your original DataFrame

# # Define the columns to include as they are
# columns_to_include = ['id', 'title', 'accords', 'rating', 'date', 'top_notes', 'middle_notes', 'base_notes', 'love', 'like', 'dislike']

# # Define the columns to check for attribute inclusion
# attributes = {
#     'season': ['autumn', 'spring', 'winter', 'summer'],
#     'time': ['day', 'night'],
#     'age': ['youth', 'adult'],
#     'gender': ['male', 'female']
# }

# # Create a new dictionary to store the data for the new DataFrame
# data_dict = {column: final_normalized_df[column] for column in columns_to_include}

# # Iterate over each attribute and check if the value is >= 0.5
# for attribute, attribute_columns in attributes.items():
#     data_dict[attribute] = final_normalized_df.apply(lambda row: [column for column in attribute_columns if row[column] >= 0.5], axis=1)

# # Create the new DataFrame
# final_dataset = pd.DataFrame(data_dict)

# # Display the new DataFrame
# print(final_dataset)


In [29]:
# Define the columns to include as they are
columns_to_include = ['id', 'title', 'accords', 'rating', 'date', 'top_notes', 'middle_notes', 'base_notes', 'love', 'like', 'dislike']

# Define the columns to check for attribute inclusion
attributes = {
    'season': ['autumn', 'spring', 'winter', 'summer'],
    'time': ['day', 'night'],
    'age': ['youth', 'adult'],
    'gender': ['male', 'female']
}

# Initialize an empty list to store data for the new DataFrame
data_rows = []

# Iterate over each row of the original DataFrame
for _, row in final_normalized_df.iterrows():
    # Create a dictionary to store data for the current row
    row_data = {column: row[column] for column in columns_to_include}
    
    # Iterate over each attribute and check if the value is >= 0.5
    for attribute, attribute_columns in attributes.items():
        # Get the attribute values for the current row
        attribute_values = [column for column in attribute_columns if row[column] >= 0.5]
        # Add the attribute values to the row data
        row_data[attribute] = list_to_comma_separated_string(attribute_values)
    
    # Append the row data to the list of data rows
    data_rows.append(row_data)

# Create the new DataFrame from the list of data rows
final_dataset = pd.DataFrame(data_rows)

# Display the new DataFrame
print(final_dataset)


       id                                              title  \
0   44152        Aamal The Spirit of Dubai for women and men   
1   19425                     Aatifa Ajmal for women and men   
2   23831           AA Al-Jazeera Perfumes for women and men   
3   49977  aarewasser Art of Scent - Swiss Perfumes for w...   
4   27823    Aaliyah Hamidi Oud & Perfumes for women and men   
..    ...                                                ...   
95  15056                 C-Thru Blooming Sarantis for women   
96  23471  Charlotte Aux Fraises Adopt' by Reserve Nature...   
97  16635                Strawberry Creme Candie's for women   
98  38744  Voodoo Love Bourbon French Parfums for women a...   
99  14350            Strawberry Starfruit Fruttini for women   

                                              accords  rating  date  \
0          woody,earthy,animalic,amber,musky,balsamic    5.00  2017   
1                  fresh spicy,woody,musky,rose,amber    4.20  2014   
2                 

In [30]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'Normalized_Data.csv'

# Save the entire DataFrame to a CSV file
final_dataset.to_csv(csv_file_path, index=False)

print(f"DataFrame data has been saved to '{csv_file_path}'.")


DataFrame data has been saved to 'Normalized_Data.csv'.
